# Download all protein sequences in orthoDB v11 for one/multiple species

1. odb11v0_OG2genes.tab.gz were downloaded from https://data.orthodb.org/download/
wget https://data.orthodb.org/download/odb11v0_OG2genes.tab.gz
2. get the OG ids that are related to oil palm (taxid 51953)
zcat odb11v0_OG2genes.tab.gz | grep 51953_0 | cut -f 1 | sort | uniq > OG_51953_0.tab
3. Download the oil palm protein sequences from the clusters that contains oil palm sequences
nohup sh download.sh > logfile 2>&1 < /dev/null &

download.sh:
#!/bin/bash
output_file="combined_data.tsv"

while IFS= read -r line; do
    curl "https://data.orthodb.org/current/fasta?id=$line&species=51953" -L -k >> "$output_file"
done < "OG_51953_0.tab"

In [ ]:
curl 'https://data.orthodb.org/current/blast?seq=MGQMGGPDGDGPHHQYHYQALLAAVQNPSQGLHVPLHAGAGAPAAGPGPRPGADADASSTHNANATPHSQPPRAFTDWSASNSAFAAQPAPATTNTPFHYNLSQSYALWTHYMLNKNVSYSTYSTPHEPLRHTHIPDKYSGCAFSLGFDSFTTMSLGPNICANMTPMERSISAKEPENSEDLPTVVRSSDEMDTRNSGDVRRDTVDTLPESKQSHESCASVSNKFDSGEYQVILRKELTKSDVANSGRIVLPKKDAEAGLPPLVQGDPLILQMDDMVLPIIWKFKYRFWPNNKSRMYILEAAGEFVKTHGPSGRGYAHYLQKLRTWQIYYPWGEVHSADNP' -L -o blast.dat

In [ ]:
!curl 'https://data.orthodb.org/current/blast?seq=MGQMGGPDGDGPHHQYHYQALLAAVQNPSQGLHVPLHAGAGAPAAGPGPRPGADADASSTHNANATPHSQPPRAFTDWSASNSAFAAQPAPATTNTPFHYNLSQSYALWTHYMLNKNVSYSTYSTPHEPLRHTHIPDKYSGCAFSLGFDSFTTMSLGPNICANMTPMERSISAKEPENSEDLPTVVRSSDEMDTRNSGDVRRDTVDTLPESKQSHESCASVSNKFDSGEYQVILRKELTKSDVANSGRIVLPKKDAEAGLPPLVQGDPLILQMDDMVLPIIWKFKYRFWPNNKSRMYILEAAGEFVKTHGPSGRGYAHYLQKLRTWQIYYPWGEVHSADNP' -L -o blast.dat

In [ ]:
!pwd

## parse the search.data

In [ ]:
import json

file_path = "/bioinfo/palm/ref/orthoDBv11/search.dat"

with open(file_path, "r") as json_file:
    data_dict = json.load(json_file)

ortholog_path = "/bioinfo/palm/ref/orthoDBv11/orthologs.dat"

with open(ortholog_path, "r") as json_file:
    ortholog_dict = json.load(json_file)

# parse the downloaded protein sequence to remove the redundent ones

In [ ]:
from Bio import SeqIO
import json

sequence_dic = {}
tag_dic = {}
level_dic = {}
with open('/bioinfo/palm/ref/orthoDBv11/oil_palm_protein.fa','w') as fh:
    for record in SeqIO.parse('/bioinfo/palm/ref/orthoDBv11/combined_data.tsv','fasta'):
        if record.id in sequence_dic:
            # gene already wrote out, check whether the sequence is different
            if record.seq != sequence_dic[record.id]:
                # need to write to fh if the sequence is different
                gene, description = record.description.split(' ', 1)
                description = description.strip()
                description_dict = json.loads(description)
                cluster = description_dict['pub_og_id']
                record.id = record.id + '_' + cluster
                tag_dic[gene] = cluster
                SeqIO.write(record, fh, 'fasta')
                print(gene) #nothing was printed, which means the protein sequences always stay the same no matter the node
        else:
            SeqIO.write(record, fh, 'fasta')
        sequence_dic[record.id] = record.seq
        

# Now a simpler code with the understanding that the protein sequences always stay the same for the same gene no matter the node

In [ ]:
from Bio import SeqIO
import json

sequence_dic = {}
level_dic = {}
with open('/bioinfo/palm/ref/orthoDBv11/oil_palm_protein.fa','w') as fh:
    for record in SeqIO.parse('/bioinfo/palm/ref/orthoDBv11/combined_data.tsv','fasta'):
        gene, description = record.description.split(' ', 1)
        description = description.strip()
        description_dict = json.loads(description)
        cluster = description_dict['pub_og_id']
        level = cluster.split('at')[1]
        if level in level_dic:
            level_dic[level].append(gene)
        else:
            level_dic[level] = [gene]
        if record.id not in sequence_dic:
            SeqIO.write(record, fh, 'fasta')
            sequence_dic[record.id] = record.seq
        
        

In [ ]:
lengths_dict = {}

# Iterate through the dictionary values and calculate lengths
for key, value in level_dic.items():
    unique_items = set(value)  # Create a set to store unique items
    lengths_dict[key] = len(unique_items)

print(lengths_dict)

This means one gene is classified into only one cluster at most at one level. 

hen we used orthomapper to map all dura proteins to orthodb v11. Results are in /bioinfo/tools/orthologer/Results

Now let's get the protein sequences that wasn't placed to any orthodb clusters (14632).

Note that if a gene was not placed in orthoDBv11 by othomapper, it won't be found in the reciprocal mapping between dura and EG5 inOrthoDBv11 either. 

Now what do we do? Map to EG5 just to see what they could have done?

In [ ]:
import pandas as pd
from Bio import SeqIO

file_path = 'dura.og.annotations'
df = pd.read_csv(file_path, sep='\t', header=0)
df.rename(columns={'#query': 'query'}, inplace=True)
df_eg5 = pd.read_csv('dura_EG5_orthoDB11.tbl', sep='\t', header=None)
i = 0
with open('dura_proteins_not_in_orthodbv11.fasta','w') as fh:
    for record in SeqIO.parse('../dura/dura_proteins.fasta','fasta'):
        if record.id not in df['query'].values:
            if record.id in df_eg5[0]:
                print(record.id)
            SeqIO.write(record, fh, 'fasta')
            i += 1
print(i)


# Fix the ID in EG5.1 cds and protein.fa

In [ ]:
from Bio import SeqIO

in_file = '/home/huan/build/orthologer/EG5.1_Genes.V3.na.fa'
out_file = '/home/huan/build/orthologer/EG5.1_Genes.V3.newid.na.fa'
with open(out_file,'w') as fh:
    for record in SeqIO.parse(in_file,'fasta'):
        record.id = record.description.split()[1].split('=')[1]
        SeqIO.write(record, fh, 'fasta')

# Merge the orthomapper results from all three levels, with priority to lower levels.

In [ ]:
import pandas as pd
from Bio import SeqIO

name = 'maize'

file1 = '~/build/orthologer/Results/{}_4447.og.annotations'.format(name)
df1 = pd.read_csv(file1, sep='\t', header=0)
df1.rename(columns={'#query': 'query'}, inplace=True)

file2 = '~/build/orthologer/Results/{}_3193.og.annotations'.format(name)
df2 = pd.read_csv(file2, sep='\t', header=0)
df2.rename(columns={'#query': 'query'}, inplace=True)

df2_new = df2[~df2['query'].isin(df1['query'])]
df12 = pd.concat([df1, df2_new], ignore_index=True)

file3 = '~/build/orthologer/Results/{}_33090.og.annotations'.format(name)
df3 = pd.read_csv(file3, sep='\t', header=0)
df3.rename(columns={'#query': 'query'}, inplace=True)

df3_new = df3[~df3['query'].isin(df12['query'])]
df123 = pd.concat([df12, df3_new], ignore_index=True)

# change their names to gene names (this only applies to EG5.1)
if name == 'EG5.1':
    header = pd.read_csv('~/build/orthologer/EG5.1_aa.header', sep=' ', header=None, names=['old','new','len'])
    header['old'] = header['old'].str[1:]
    header['new'] = header['new'].str.split('=').str[1]
    header.drop('len', axis=1, inplace=True)
    dictionary = {key: value for key, value in zip(header['old'], header['new'])}
    df123['query'] = df123['query'].map(dictionary)

df123.to_csv('~/build/orthologer/{}_orthoDBv11_4447_3193_33090.og.annotations'.format(name), sep='\t', index=False)

# dura gene-> og -> at gene

In [ ]:
df1 = pd.read_csv('/home/huan/build/orthologer/EG5.1_orthoDBv11_4447_3193_33090.og.annotations', sep='\t', header=0)
df1.rename(columns={'#query': 'query'}, inplace=True)

df2 = pd.read_csv('/home/huan/build/orthologer/maize_orthoDBv11_4447_3193_33090.og.annotations', sep='\t', header=0)
df2.rename(columns={'#query': 'query'}, inplace=True)

df3 = pd.read_csv('/home/huan/build/orthologer/rice_orthoDBv11_4447_3193_33090.og.annotations', sep='\t', header=0)
df3.rename(columns={'#query': 'query'}, inplace=True)

df4 = pd.read_csv('/home/huan/build/orthologer/Arabidopsis_orthoDBv11_4447_3193_33090.og.annotations', sep='\t', header=0)
df4.rename(columns={'#query': 'query'}, inplace=True)

df5 = pd.read_csv('/home/huan/build/orthologer/dura_orthoDBv11_4447_3193_33090.og.annotations', sep='\t', header=0)
df5.rename(columns={'#query': 'query'}, inplace=True)

In [ ]:
df5 = pd.read_csv('/home/huan/build/orthologer/dura_orthoDBv11_4447_3193_33090.og.annotations', sep='\t', header=0)
df5.rename(columns={'#query': 'query'}, inplace=True)

In [ ]:
gff = pd.read_csv('/home/huan/build/orthologer/Araport11_GFF3_genes_transposons.symbol.gff.2', sep='\t', header=None, comment="#", names=['chr','source','type','start','end','score','strand','phase','attributes'])
gff_mRNA = gff[gff['type']=='mRNA']

# Apply the function to each row in the 'attribute' column
parsed_attributes = gff_mRNA['attributes'].apply(parse_attribute).apply(pd.Series)

# Join the parsed attributes back to the original DataFrame
gff_mRNA = gff_mRNA.join(parsed_attributes)


In [ ]:
gff_light = gff_mRNA[['symbol','ID']]
gff_light.rename(columns={'ID': 'query'}, inplace=True)
df4_merged = pd.merge(df4, gff_light, on='query',how='inner')
df4_light = df4_merged[['ODB_OG','symbol']].drop_duplicates()
df4_light = df4_light[df4_light['symbol'] != 'None']
df4_light = df4_light.groupby('ODB_OG')['symbol'].agg(lambda x: ';'.join(x.astype(str))).reset_index()
df1_merged = pd.merge(df1, df4_light, on='ODB_OG',how='inner')

In [ ]:
df5_merged = pd.merge(df5, df4_light, on='ODB_OG',how='inner')

In [ ]:
df4_light.to_csv('/home/huan/build/orthologer/Arabidopsis_OG_symple.map')

In [ ]:
df1_merged.to_csv('/home/huan/build/orthologer/EG5.1_orthoDBv11_4447_3193_33090.og.symbol.annotations', index=False, sep='\t')

In [ ]:
df5_merged.to_csv('/home/huan/build/orthologer/dura_orthoDBv11_4447_3193_33090.og.symbol.annotations', index=False, sep='\t')

In [ ]:
gff = pd.read_csv('/home/huan/build/orthologer/EG5.1_Genes.V3.newid.gff3', sep='\t', header=None, comment="#", names=['chr','source','type','start','end','score','strand','phase','attributes'])
gff_gene = gff[gff['type']=='gene']
parsed_attributes = gff_gene['attributes'].apply(parse_attribute).apply(pd.Series)

# Join the parsed attributes back to the original DataFrame
gff_gene = gff_gene.join(parsed_attributes)

# merge snpEff files from different chromosomes

In [ ]:
import glob
# Define the pattern to match your TSV files
file_pattern = "/home/huan/build/orthologer/EG5.1_batch2_190_chr*.snpEff.genes.txt"

# Use glob to get a list of file paths
file_paths = glob.glob(file_pattern)

# List to hold each DataFrame
dfs = []

# Loop through the file paths, read each file into a DataFrame, and add it to the list
for file_path in file_paths:
    df = pd.read_csv(file_path, sep='\t', header = 1)
    dfs.append(df)

# Concatenate all DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)
combined_df.rename(columns={'#GeneName': 'GeneName'}, inplace=True)

# Check the combined DataFrame
combined_df.to_csv('/home/huan/build/orthologer/EG5.1_batch2_190.snpEff.genes.csv', index = False)


In [ ]:
plant_hormone_gene_list = ['TIR1','Aux','IAA','ARF','YUCCA','PIN','GAI','SPY','GA20ox','GA3ox','GA2ox','ARR','CRE1','WOL','AHK4','ETR1','CTR1','EIN2','PYR','PYL','RCAR','ABI','BRI1','BIN2','BZR1','COI1','JAZ','LOX', 'AOS', 'AOC','MAX1','D14','D53','SMXLs','NPR1']
MADS_gene_list = ['AG','AP1','AP3','PI','SEP1','SOC1','FLC','SVP','SHP2','SEP3']
homeobox_gene_list = ['WUS','KNAT','GL2','WOX','HB-','BEL1','PHB','ATH1']
phyto_list = ['PHYA','PHYB','PHYC','PHYD','PHYE','PIF','PIL']
#PIF
gene_list = phyto_list.copy()
outbed = '/home/huan/build/orthologer/EG5.1_PIF.bed'
outfile = '/home/huan/build/orthologer/PIF.gene.list'
# Plant hormone
#gene_list = plant_hormone_gene_list.copy()
#outbed = '/home/huan/build/orthologer/EG5.1_plant_hormone.bed'
#outfile = '/home/huan/build/orthologer/plant_hormone.gene.list'
# MADS-box
#gene_list = MADS_gene_list.copy()
#outbed = '/home/huan/build/orthologer/EG5.1_MADS_box.bed'
#outfile = '/home/huan/build/orthologer/MADS_box.gene.list'
# homeobox
#gene_list = homeobox_gene_list.copy()
#outbed = '/home/huan/build/orthologer/EG5.1_homeobox.bed'
#outfile = '/home/huan/build/orthologer/Homeobox.gene.list'

snpEff = pd.read_csv('/home/huan/build/orthologer/EG5.1_batch2_190.snpEff.genes.csv')
snpEff['ID'] =  snpEff['TranscriptId'].str[:-2]
snpEff.columns = [col.lstrip('variants_') if col.startswith('variants_') else col for col in snpEff.columns]
out_snp = outbed + '.snpEff.csv'
with open(outbed,'w') as fh_bed:
    with open(outfile, 'w') as fh_file:
        fh_file.write(','.join(['Gene', 'OG_ID', 'EG5.1', 'Dura', 'Arabidopsis', 'Rice', 'Maize']) + '\n')
        for gene in gene_list:
            ogs = df4_light[df4_light['symbol'] == gene]['ODB_OG']
            if ogs.shape[0] == 0:
                pattern = rf'{gene}[1-9][0-9]*'
                ogs = df4_light[df4_light['symbol'].str.match(pattern)]['ODB_OG']    
            for og in ogs:
                genes = df4_light[df4_light['ODB_OG'] == og]['symbol']
                for gene in genes:
                    EG_genes = df1[df1['ODB_OG'] == og]['query']
                    gff_sub = gff_gene[gff_gene['ID'].isin(EG_genes)]
                    EG_genes = ';'.join(EG_genes)
                    EG_genes = ';'.join(df1[df1['ODB_OG'] == og]['query'])
                    maize_genes = ';'.join(df2[df2['ODB_OG'] == og]['query'])
                    rice_genes = ';'.join(df3[df3['ODB_OG'] == og]['query'])
                    at_genes = ';'.join(df4[df4['ODB_OG'] == og]['query'])
                    dura_genes = ';'.join(df5[df5['ODB_OG'] == og]['query'])
                    fh_file.write(','.join([gene, og, EG_genes, dura_genes, at_genes, rice_genes, maize_genes]) + '\n')
                    for index, row in gff_sub.iterrows():
                        fh_bed.write('\t'.join([row['chr'],str(row['start']),str(row['end']),row['ID'], gene, og]) + '\n')

bed = pd.read_csv(outbed, sep = '\t', header = None, names= ['chr','start','end','ID','gene','og'])
bed_merge = pd.merge(bed, snpEff, on='ID',how='inner').drop(['GeneName','GeneId'], axis=1)
bed_merge.to_csv(out_snp, index = False)

# MADS-box genes


In [ ]:
anno = pd.read_csv('/home/huan/build/orthologer/EG5.1_orthoDBv11_4447_3193_33090.og.annotations', sep = '\t')
filtered_df = anno[anno['Description'].str.contains('MADS-box')]
IDs = [i.split(',') for i in filtered_df['Interpro']]
EG5_ID = set([item for sublist in IDs for item in sublist])
anno = pd.read_csv('/home/huan/build/orthologer/dura_orthoDBv11_4447_3193_33090.og.annotations', sep = '\t')
filtered_df = anno[anno['Description'].str.contains('MADS-box')]
IDs = [i.split(',') for i in filtered_df['Interpro']]
dura_ID = set([item for sublist in IDs for item in sublist])
anno = pd.read_csv('/home/huan/build/orthologer/rice_orthoDBv11_4447_3193_33090.og.annotations', sep = '\t')
filtered_df = anno[anno['Description'].str.contains('MADS-box')]
IDs = [i.split(',') for i in filtered_df['Interpro']]
rice_ID = set([item for sublist in IDs for item in sublist])
anno = pd.read_csv('/home/huan/build/orthologer/maize_orthoDBv11_4447_3193_33090.og.annotations', sep = '\t')
filtered_df = anno[anno['Description'].str.contains('MADS-box')]
IDs = [i.split(',') for i in filtered_df['Interpro']]
maize_ID = set([item for sublist in IDs for item in sublist])
anno = pd.read_csv('/home/huan/build/orthologer/Arabidopsis_orthoDBv11_4447_3193_33090.og.annotations', sep = '\t')
filtered_df = anno[anno['Description'].str.contains('MADS-box')]
IDs = [i.split(',') for i in filtered_df['Interpro']]
AT_ID = set([item for sublist in IDs for item in sublist])

In [ ]:
union = list(set(EG5_ID) | set(dura_ID) | set(rice_ID) | set(maize_ID) | set(AT_ID))

# MADS-box interpro IDs are: ['IPR033897','IPR002487','IPR002100','IPR031050','IPR036879','IPR033896']

In [ ]:
a='MGRGRVQLKRIENKINRQVTFSKRRAGLLKKAHEISVLCDAEVALVVFSHKGKLFEYSTDSCMEKILERYERYSYAERQLIAPESDVNTNWSMEYNRLKAKIELLERNQRHYLGEDLQAMSPKELQNLEQQLDTALKHIRTRKNQLMYESINELQKKEKAIQEQNSMLSKQIKEREKILRAQQEQWDQQNQGHNMPPPLPPQQHQIQHPYMLSHQPSPFLNMGGLYQEDDPMAMRRNDLELTLEPVYNCNLGCFAA'

In [ ]:
dic = {}
with open('/home/huan/build/orthologer/IPR002100.gene') as fh:
    for line in fh:
        dic[line.rstrip()] = ''
with open('/home/huan/build/orthologer/dura_IPR002100_proteins.fasta','w') as outfh:
    for record in SeqIO.parse('/home/huan/build/orthologer/dura_proteins.fasta','fasta'):
        if record.id in dic:
            SeqIO.write(record, outfh, 'fasta')
        

In [ ]:
dic

In [ ]:
print(a1, a2, a3)

In [ ]:
len('SDVNTNWSMEYNRLKAKIELLERNQRHYLGEDLQAMSPKELQNLEQQLDTALKHIRTRKNQLMYESINELQKKEKAIQEQNSMLSKQI')

In [ ]:
a[:172]

In [ ]:
list_of_lists = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flattened_list = [item for sublist in list_of_lists for item.spli in sublist]

In [ ]:
flattened_list

In [ ]:
# OK, now I need to write a script where it takes a symbol, and returns the gene name i

In [ ]:
import pandas as pd

# Example DataFrame
df1 = pd.DataFrame({
    'attribute': ['ID=AT1G01010.1;Parent=AT1G01010;Name=AT1G01010.1']
})


# Apply the function to each row in the 'attribute' column
parsed_attributes = df1['attribute'].apply(parse_attribute).apply(pd.Series)

# Join the parsed attributes back to the original DataFrame
df1 = df1.join(parsed_attributes)

print(df1)


In [ ]:
union = set(df2['ODB_OG']) | set(df3['ODB_OG']) | set(df4['ODB_OG'])

In [ ]:
len(union)

In [ ]:
shared_og_all = set(df1['ODB_OG']) & union

In [ ]:
len(shared_og_all)

In [ ]:
shared_og = set(df1['ODB_OG']) & set(df2['ODB_OG'])

In [ ]:
len(shared_og)

In [ ]:
len(shared_og)/23056

In [ ]:
len(df1['ODB_OG'].unique())

In [ ]:
df2.shape

In [ ]:
12593/15674

# Now we need to go from gene name to clusterID, then dura gene ID.
Play with the API

In [ ]:
import subprocess, json

gene_name = 'ABI5'
level = 4447
cmd = "curl 'https://data.orthodb.org/current/search?query={}&level={}' -L -o {}_at{}.dat".format(gene_name, level, gene_name, level)
print(cmd)
subprocess.run(cmd, shell=True, check=True)
with open("{}_at{}.dat".format(gene_name, level), "r") as json_file:
    data_dict = json.load(json_file)
anno_file = 'dura_orthoDBv11_4447_3193_33090.og.annotations'
df = pd.read_csv(anno_file, sep='\t', header=0)
selected_rows = df[df['ODB_OG'].isin(data_dict['data'])]
selected_rows.rename(columns={'#query': 'query'}, inplace=True)
i = 0
protein_file = '/bioinfo/palm/ref/dura/dura_proteins.fasta'
with open('dura_proteins_{}_at{}.aa'.format(gene_name, level),'w') as fh:
    for record in SeqIO.parse(protein_file,'fasta'):
        if record.id in selected_rows['query'].values:
            cluster = selected_rows.loc[selected_rows['query'] == record.id, 'ODB_OG']
            tag = selected_rows.loc[selected_rows['query'] == record.id, 'Description']
            if len(tag) == 1:
                record.description = ':'.join([cluster.iloc[0], tag.iloc[0]])
            else:
                print(record.id + 'appeared in more than one cluster?')

            SeqIO.write(record, fh, 'fasta')
            i += 1
print(i)

In [ ]:
len(selected_rows.loc[selected_rows['query'] == 'Egu023084-mRNA-1', 'ODB_OG'])

# We ran orthoDB on four other species and compared them

In [ ]:
import pandas as pd
from collections import Counter
level = 4447
def og_dic(level, name):
    # this function takes a orthoDB annotation, count the cluser compotion and return a pd
    anno_file = "/bioinfo2/palm/ref/orthoDBv11/{}_{}.og.annotations".format(name, level)
    df = pd.read_csv(anno_file, sep='\t', header=0)
    dic = Counter(df['ODB_OG'])
    # Convert dictionaries to DataFrames
    df = pd.DataFrame(list(dic.items()), columns=['key', name]).set_index('key')
    return(df)

df_dura = og_dic(4447, 'dura')
df_coco = og_dic(4447, 'coco')
df_date = og_dic(4447, 'date')
df_picifera = og_dic(4447, 'EG5')
df_rice = og_dic(4447, 'rice')

df = pd.concat([df_dura, df_coco, df_date, df_picifera, df_rice], axis=1)

df.to_csv("/bioinfo2/palm/ref/orthoDBv11/combined_og.csv")


In [ ]:
df

In [ ]:
duplicates = df123[df123.duplicated('query')]  # Replace 'column_name' with the actual column name


In [ ]:
import pandas as pd
anno_file = 'dura_orthoDBv11_4447_3193_33090.og.annotations'
df = pd.read_csv(anno_file, sep='\t', header=0)
tpm_file = 'counts_transcript_TPM_concise.txt'
tpm = pd.read_csv(tpm_file, sep=',', header=0)


In [ ]:
df_1 = df[['query','Description']]
df_2 = pd.merge(df_1, tpm, on='query',how='outer')

In [ ]:
df_2.to_csv('output.csv', index=False)

# Transfer Dura og to EG5.1 og.

In [ ]:
import pandas as pd
# Function to parse the 'Attributes' column in gff file and convert it into a dictionary of key-value pairs
def parse_attributes(attribute_str):
    attribute_pairs = attribute_str.split(';')
    attribute_dict = {}
    for pair in attribute_pairs:
        if len(pair.split('=')) == 2:
            key, value = pair.split('=')
        else:
            key = pair.split('=')[0]
            value = ''
        attribute_dict[key] = value
    return attribute_dict

def filtered_rows2bedfile(filtered_rows, bedfile):
    # takes a list of gene ids and write their regions into a bed file
    df = filtered_rows[['Seqid', 'Start','End','ID']]
    df_no_duplicates = df.drop_duplicates()
    df_sorted = df_no_duplicates.sort_values(by=['Seqid', 'Start'])
    # add pound(#) to the first column name - convention of bed file
    df_sorted.columns = ['#' + df_sorted.columns[0] if i == 0 else col for i, col in enumerate(df_sorted.columns)]
    df_sorted.to_csv(bedfile, sep='\t', index=False)
    return df_sorted

# read in the Dura annotation
dura_df = pd.read_csv('/home/huan/build/orthologer/dura_orthoDBv11_4447_3193_33090.og.annotations', sep='\t')
# read in the EG5.1 annotation
EG51_df = pd.read_csv('/home/huan/build/orthologer/EG5.1_orthoDBv11_4447_3193_33090.og.annotations', sep='\t')
## read the gff file
gff_file = '/home/huan/Documents/palm/EG5.1_Genes.V3.gff3'
gff_column_names = ['Seqid', 'Source', 'Type', 'Start', 'End', 'Score', 'Strand', 'Phase', 'Attributes']
df = pd.read_csv(gff_file, sep='\t',comment='#', names=gff_column_names)
df1 = df['Attributes'].apply(parse_attributes).apply(pd.Series)
df_gff = pd.concat([df.drop('Attributes', axis=1), df1], axis=1)

trait_list = ['drought','height','fa','fattyacid','fruit','SAUR']    

for trait in trait_list:
    # read in the catalog file
    cat_file = '/home/huan/build/orthologer/dura_{}_catalogue.txt'.format(trait)
    cat_df = pd.read_csv(cat_file, sep='\t', header=None, names=['query','gene'])
    # cat_dura_df = dura_df[dura_df['query'].isin(cat_df['query'])]
    cat_dura_df = pd.merge(cat_df, dura_df, on='query',how='inner')
    cat_dura_df.rename(columns={'query': 'dura_gene'}, inplace=True)
    # sorted_df = cat_dura_df.sort_values(by='ODB_OG')
    cat_EG51_df = EG51_df[EG51_df['ODB_OG'].isin(cat_dura_df['ODB_OG'])]
    shared_columns = set(EG51_df.columns).intersection(set(cat_dura_df.columns))
    columns_to_keep = ['ODB_OG'] + [col for col in cat_dura_df.columns if col not in shared_columns]
    cat_dura_df_reduced = cat_dura_df[columns_to_keep]
    
    # Perform the merge
    cat_EG51_df = pd.merge(cat_dura_df_reduced, EG51_df, on='ODB_OG', how='inner')
    selected_df = cat_EG51_df[['ODB_OG', 'gene', 'dura_gene', 'query']]
    selected_df.rename(columns={'query': 'EG5.1_gene', 'gene': 'gene_name'}, inplace=True)
    
    selected_df.to_csv('~/build/orthologer/EG5.1_{}_catalogue.txt'.format(trait), sep='\t', index=False)\
    
    
    
    ## read in the orthoDB annotation file
    selected_rows = EG51_df[EG51_df['ODB_OG'].isin(selected_df['ODB_OG'])]
    selected_rows = selected_rows.rename(columns={'#query': 'query'})
    
    
    # prepare bed file gene region
    filtered_rows = df_gff[df_gff['ID'].isin(selected_rows['query'])]
    bedfile = '~/build/orthologer/EG5.1_{}_exon.bed'.format(trait)
    filtered_rows2bedfile(filtered_rows, bedfile) 

# prepare bed file for exons only: 9 should be enough? No it is not. can easily go beyond that.
# pattern = '|'.join([f"{item}:[1-9]$" for item in selected_rows['query']])
# pattern = r'^(?:' + '|'.join(selected_rows['query']) + r'):\d+$'

# Filtering rows
# filtered_rows = df_gff[df_gff['ID'].str.contains(pattern, regex=True)]
# bedfile = gene_name + '_at' + level + '_dura.exon.bed'
# filtered_rows2bedfile(filtered_rows, bedfile) 

# Synteny

In [ ]:
colnames = ['pic_chr','pic_len','p_start','p_end','orientation','dur_chr','dur_len','d_start','d_end','A','B','C','D','E','AS','nn','tp','cm','s1','s2','df','zd','rl','cg']
syn_chr1 = pd.read_csv('/home/huan/Documents/palm/pic_chr1_dura.paf', sep='\t', names=colnames)
df_sorted = syn_chr1.sort_values(by='p_start')















# misssed jobs in freebayes?

In [ ]:
import bisect
i = 0
dic = {}
with open('/home/huan/Downloads/EG5.1_batch2_190_chr1.pos') as fh:
    for line in fh:
        j = int(line.rstrip())
        dis = j-i
        dic[i] = dis
        i = j
i = 0
sorted_keys = sorted(dic.keys())

with open('/home/huan/Downloads/EG5.1_batch2_262_chr1.pos') as fh:
    for line in fh:
        j = int(line.rstrip())
        if j - i > 10000:
            if i in dic:
                if dic[i] < 10000:
                    print(i,j)
            else:
                # Find the left and right indices
                left_index = bisect.bisect_left(sorted_keys, i)
                right_index = bisect.bisect_right(sorted_keys, j)
                # Calculate the count
                count = right_index - left_index
                if count > 1:
                    print(i,j)
                    break
        i = j
        

# From gff to bed (using ID instead of transciptID)

In [ ]:
# Function to parse the attribute column
import pandas as pd
import os

def parse_attribute(attribute):
    pairs = attribute.split(';')
    attr_dict = {}
    for pair in pairs:
        # Check if the element contains more attributes separated by ','
        if pair.count('=') > 1:
            if ',' in pair:
                sub_pairs = pair.split(',')
                for sub_pair in sub_pairs:
                    if '=' in sub_pair:
                        print(sub_pair)
                        key, value = sub_pair.split('=')
                        attr_dict[key] = value
        else:
            if '=' in pair:
                key, value = pair.split('=')
                attr_dict[key] = value
    return attr_dict

def gff2bed(gff_file, type = None, bed_level = 'ID'):
    gff = pd.read_csv(gff_file, sep='\t', header=None, comment="#", names=['chr','source','type','start','end','score','strand','phase','attributes'])
    if type:
        gff = gff[gff['type']==type]
    # Apply the function to each row in the 'attribute' column
    parsed_attributes = gff['attributes'].apply(parse_attribute).apply(pd.Series)    
    # Join the parsed attributes back to the original DataFrame
    gff = gff.join(parsed_attributes)
    # note that gff is 1-based and bed is 0-based, so start needs to be adjusted
    gff['start'] = gff['start'].astype(int) - 1
    bed = gff[['chr','start','end',bed_level]].copy()
    bed.rename(columns={'chr': '#chr'}, inplace=True)
    root, ext = os.path.splitext(gff_file)
    bed.to_csv(root+'.bed', sep = '\t', index=False)
def idlist2seqfile(ids, seqfile, outfile):
    from Bio import SeqIO
    ids = set(ids)
    with open(outfile, 'w') as out_fh:
        for record in SeqIO.parse(seqfile, 'fasta'):
            if record.id in ids:
                SeqIO.write(record, out_fh, 'fasta')

In [ ]:
gff2bed('/home/huan/Documents/Palm/EG5.1_p5.00_sc00060_p0019.gff3')

In [ ]:
gff2bed('/home/huan/Documents/Palm/dura_Egu023247.gff')

# Get rRNA genes from EG5

In [ ]:
gff = pd.read_csv('/home/huan/Documents/Palm/genomic.gff', sep='\t', header=None, comment="#", names=['chr','source','type','start','end','score','strand','phase','attributes'])
gff_rRNA = gff[gff['type']=='rRNA']

# Apply the function to each row in the 'attribute' column
parsed_attributes = gff_rRNA['attributes'].apply(parse_attribute).apply(pd.Series)

# Join the parsed attributes back to the original DataFrame
gff_rRNA = gff_rRNA.join(parsed_attributes)
gff_rRNA['id'] = gff_rRNA['ID'].strsplit('-')[1]
                
seqfile = '/home/huan/Documents/Palm/dura_proteins.fasta'
outfile = '/home/huan/Documents/Palm/dura_30unique_proteins.fasta'
ids = idfile2seqfile(idfile,seqfile, outfile)

In [ ]:
gff_rRNA['id'] = gff_rRNA['ID'].str.split('-').str[1]

In [ ]:
gff_rRNA